In [1]:
# 기존 데이터를 분석하는 코드
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

# 한글 폰트 설정
font_path = "C:/Windows/Fonts/malgun.ttf"  # 폰트 경로를 시스템에 맞게 설정
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

# 패키지 내의 파일 경로 가져오기
test_csv_path = os.path.join(os.getcwd(), '..', '..', 'public', 'AI_bokji_result_final.csv')
try:
    df = pd.read_csv(test_csv_path, encoding='euc-kr', index_col=0)
except UnicodeError:
    df = pd.read_csv(test_csv_path, encoding='utf-8', index_col=0)

print("File loaded.")
result_csv_path = os.path.join(os.path.dirname(test_csv_path), 'AI_bokji_result_answers.csv')

File loaded.


### 기존 csv파일 전처리 및 답변 분리 ###

In [ ]:
# STT 전처리 - 앞뒤 공백 및 줄바꿈 문자 제거
df['STT_TRNF_RSLT_CN'] = df['STT_TRNF_RSLT_CN'].str.strip()

# 1. 'VOC_SDRC_DCD' 값이 'RX'인 데이터만 추출
df_rx = df[df['VOC_SDRC_DCD'] == 'RX'].copy()

# 2. 'STT_TRNF_RSLT_CN' 값 분석 (공백 및 줄바꿈 문자 제거)
df_rx['STT_TRNF_RSLT_CN'] = df_rx['STT_TRNF_RSLT_CN'].str.strip()

# 3. 중복된 데이터의 수 계산
stt_counts = df_rx['STT_TRNF_RSLT_CN'].value_counts().reset_index()
stt_counts.columns = ['STT_TRNF_RSLT_CN', 'count']

# 4. 개수가 1개인 데이터는 '기타' 항목으로 분류
def categorize_stt(row):
    if row['STT_TRNF_RSLT_CN'].startswith('눌러야') or row['STT_TRNF_RSLT_CN'].startswith('눌라야')\
            or row['STT_TRNF_RSLT_CN'].startswith('놀라야') or '녹음' in row['STT_TRNF_RSLT_CN']\
            or '번호' in row['STT_TRNF_RSLT_CN'] or '남기' in row['STT_TRNF_RSLT_CN']:
                return '기타'
    else:
        return row['STT_TRNF_RSLT_CN']

stt_counts['STT_TRNF_RSLT_CN'] = stt_counts.apply(categorize_stt, axis=1)

# 최종 결과를 데이터프레임으로 다시 정리
final_counts = stt_counts.groupby('STT_TRNF_RSLT_CN')['count'].sum().reset_index()

# Count 기준 내림차순 정렬
final_counts = final_counts.sort_values(by='count', ascending=False)

# 시각화
plt.figure(figsize=(10, 8))
plt.barh(final_counts['STT_TRNF_RSLT_CN'], final_counts['count'], color='skyblue')
plt.xlabel('Count')
plt.ylabel('STT_TRNF_RSLT_CN')
plt.title('STT_TRNF_RSLT_CN Count')
plt.gca().invert_yaxis()  # y축을 뒤집어 큰 값이 위로 오도록 설정

# 이미지 파일로 저장
image_path = os.path.join(os.path.dirname(test_csv_path), 'result.png')
plt.savefig(image_path, bbox_inches='tight')
plt.close()

# 최종 결과를 CSV 파일로 저장
final_counts.to_csv(result_csv_path, index=False, encoding='euc-kr')
print("파일이 성공적으로 저장되었습니다.", image_path)


### 분리한 답변에서 유의미한 데이터 추출

In [2]:
final_df = pd.read_csv(result_csv_path, encoding='euc-kr')

# STT_TRNF_RSLT_CN 데이터의 길이가 5 이하인 데이터 제거
filtered_df = final_df[final_df['STT_TRNF_RSLT_CN'].str.len() > 5]

# 새로운 파일로 저장
filtered_csv_path = os.path.join(os.path.dirname(result_csv_path), 'AI_bokji_result_final_filtered.csv')
filtered_df.to_csv(filtered_csv_path, index=False, encoding='euc-kr')

# 데이터 'ㄱ'부터 시작하여 오름차순으로 정렬
filtered_df = filtered_df.sort_values(by='STT_TRNF_RSLT_CN')

print("파일이 성공적으로 저장되었습니다.", filtered_csv_path)

파일이 성공적으로 저장되었습니다. C:\Users\A\Desktop\repo\PM4_JS\src\_non-module\..\..\public\AI_bokji_result_final_filtered.csv


### 이전답변을 반영하여 답변 데이터만을 저장 ###

In [4]:
answers = pd.DataFrame(columns=["Question", "Answer"])
for i in range(1, len(df)):
    current = df[i]
    before = df[i-1]
    if current["VOC_SDRC_DCD"] == 'RX'\
        and before["VOC_SDRC_DCD"] == 'TX'\
        and len(current['STT_TRNF_RSLT_CN']) >= 5:
        answer = pd.DataFrame({"Question": before["STT_TRNF_RSLT_CN"].replace(" ", ""),
                               "Answer": current["STT_TRNF_RSLT_CN"]})
        answers = pd.concat([answers, answer], ignore_index=True)

answers.to_csv(result_csv_path, index=False, encoding='euc-kr')



KeyError: 1